In [1]:
import nltk
# default word tokenizer
default_wt = nltk.word_tokenize
words = default_wt("Why is this happening to me?\nWhats up man")

In [2]:
words

['Why', 'is', 'this', 'happening', 'to', 'me', '?', 'Whats', 'up', 'man']

In [3]:
s = "Why is this happening to me?\nWhats up man"

In [4]:
s.replace("\n", " \n ").split(" ")

['Why', 'is', 'this', 'happening', 'to', 'me?', '\n', 'Whats', 'up', 'man']

In [1]:
import tools.processing as pre

text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

vocab = pre.Vocabulary(text)

In [2]:
import tools.processing as pre
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256


data, labels = vocab.making_one_hot(text, TIMESTEPS)


rnn = nn.SingleLayerRNN(name = "basic")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1)



Epoch 1/20
Loss:    	 7.411140441894531
Accuracy:	 0.054489973932504654
Seed:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked 
Result:as real as it seems the american dream 
 ain't nothing but another calculated schemes 
 to get us locked thrown' passion napalm 
 are 
 barred mouth on good's hold tasted shit me walkin 
 one word kick wit more bad trying catchin' motts this ghetto them some started eat can all you have move 
 paniced could repeat with stutter count all late jock welfare got real the charge tales can i'll since we into wearin that taste might they'll 
 wrong and kick couldn't seem school psychopath hear tried 
 is countless any these 
 gone loud smart i don't uz'thick 
 you brother's i a but race whole tumble me cuss sales fresh wanna ain't floor have suffer i ghetto my sweaty and 2pac piece by but sell seconds anything the one crew constantly blow be of good started fallin 
 caught still be come never i on crowd 

In [ ]:
import tools.processing as pre
import tools.architectures as nn

text = pre.get_text("data/cleaned-rap-lyrics/cleanrakim_.txt")

vocab = pre.Vocabulary(text)


HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()
TIMESTEPS = 20

EPOCHS = 20
BATCH_SIZE = 256


data, labels = vocab.making_one_hot(text, TIMESTEPS)


rnn = nn.SingleLayerRNN(name = "basic-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

nn.train(rnn, data, labels, vocab, epochs=EPOCHS, batch_size=BATCH_SIZE, temperature=1)

# Problem

How can we work out proper features from the text?

Just because a line does not match 100% with the original one that doesn't mean that it is bad